In [1]:
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

import theano.tensor as tt
import pymc3 as pm

import exoplanet as xo
from specgp.terms import KronTerm
from specgp.means import KronMean
from specgp.distributions import MvUniform

Start by loading the traces from our MCMC analysis of the SOHO data:

In [2]:
with pm.Model() as model:
    trace = pm.load_trace('traces/.pymc_3.trace')
    mu, sig = {}, {}
    for k, v in trace[0].items():
        vals = trace.get_values(k)
        mu[k] = np.mean(vals, axis=0)
        sig[k] = np.std(vals, axis=0)

We'll use these values to simulate sun-like light curves:

In [3]:
term1 = xo.gp.terms.SHOTerm(log_S0=mu["logS0"][0], 
                            log_w0=mu["logw"][0], 
                            log_Q=-np.log(np.sqrt(2))
                            )
term2 = xo.gp.terms.SHOTerm(log_S0=mu["logS0"][1], 
                            log_w0=mu["logw"][1], 
                            log_Q=mu["logQ"]
                            )
alpha1 = np.exp([0, mu["alpha1"][0], mu["alpha2"][0]])
alpha2 = np.exp([0, mu["alpha1"][1], mu["alpha2"][1]])
kernel = (KronTerm(term1, alpha=alpha1) + 
          KronTerm(term2, alpha=alpha2))

t = np.linspace(-1, 1, 1000)
logsig = np.array([-4]*3)
diag = np.exp(2*logsig)[:, None] * np.ones((3, len(t)))
mean = mu["mean"][:, None] * np.ones_like(t)
gp = xo.gp.GP(x=t, kernel=kernel, diag=diag, mean=KronMean(mean), J=4)

generate and plot simulated data!

In [ ]:
n = np.random.randn(3*len(t), 1)
y = gp.dot_l(n).eval()
r, g, b = [y[i::3] for i in range(3)]

colors = ['#FE4365', '#3F9778', '#00A9FF', '#ECA25C', '#005D7F']
pl.plot(t, r, '-', color=colors[0])
pl.plot(t, g + 1, '-', color=colors[1])
pl.plot(t, b + 2, '-', color=colors[2])

Add a transit with ``starry``:

In [ ]:
orbit = xo.orbits.KeplerianOrbit(period=5.0)
u = [0.3, 0.2]
planet = (
    xo.LimbDarkLightCurve(u)
    .get_light_curve(orbit=orbit, r=0.009, t=t, texp=0.02)
    .eval()
) * 1e3
moon = (
    xo.LimbDarkLightCurve(u)
    .get_light_curve(orbit=orbit, r=0.002, t=t+0.25, texp=0.02)
    .eval()
) * 1e3

In [ ]:
fig, ax = pl.subplots(1, 2, figsize=(15, 5))

ax[0].plot(t, r + planet + moon, '-', color=colors[0])
ax[0].plot(t, g+1 + planet + moon, '-', color=colors[1])
ax[0].plot(t, b+2 + planet + moon, '-', color=colors[2])
ax[0].set_ylim(-1, 3)

ax[1].plot(t, np.mean([r, g, b], axis=0)+1 + planet, '-', color='k')
ax[1].set_ylim(-1, 3)

[x.set_ylabel('flux') for x in ax]
[x.set_xlabel('time (days)') for x in ax]

Let's do inference on each of these separately and see how the results compare. First, the multiband light curve: 

In [ ]:
mean = (planet + moon)
mean = np.tile(mean, (1, 3))
mean = np.reshape(mean, (3*len(t), 1))
obs = y + mean

with pm.Model() as model:
    
    logS0 = pm.MvNormal("logS0", mu=mu["logS0"], 
                        chol=np.diag(sig["logS0"]), shape=2)
    logw = pm.MvNormal("logw", mu=mu["logw"], 
                       chol=np.diag(sig["logw"]), shape=2)
    logQ = pm.Normal("logQ", mu=mu["logQ"], sd=sig["logQ"])
    alpha1 =  pm.MvNormal("alpha1", mu=mu["alpha1"], 
                          chol=np.diag(sig["alpha1"]), shape=2)
    alpha2 =  pm.MvNormal("alpha2", mu=mu["alpha2"], 
                          chol=np.diag(sig["alpha2"]), shape=2)
    logsig = pm.MvNormal("logsig", mu=mu["logsig"], 
                         chol=np.diag(sig["logsig"]), shape=3)
    mean = pm.MvNormal("mean", mu=mu["mean"], 
                       chol=np.diag(sig["mean"]), shape=3)
    u = MvUniform("u", lower=[0, 0], upper=[1, 1], testval=[0.5, 0.5])
    logr = pm.Uniform("logr", lower=-20.0, upper=0.0, testval=np.log(0.01))
    t0 = pm.Uniform("t0", lower=-1, upper=1, testval=0.0)
    
    orbit = xo.orbits.KeplerianOrbit(period=5.0)
    lc = (xo.LimbDarkLightCurve(u)
          .get_light_curve(orbit=orbit, r=tt.exp(logr), t=t-t0, texp=0.02)
          *1e3)
    mean = mean[:, None] + lc.T[0]
    
    term1 = xo.gp.terms.SHOTerm(
        log_S0 = logS0[0],
        log_w0 = logw[0],
        log_Q = -np.log(np.sqrt(2))
    )
    term2 = xo.gp.terms.SHOTerm(
        log_S0 = logS0[1],
        log_w0 = logw[1],
        log_Q = logQ
    )
        
    a1 = tt.exp(tt.stack([0.0, alpha1[0], alpha2[0]]))
    a2 = tt.exp(tt.stack([0.0, alpha1[1], alpha2[1]]))
        
    kernel = (KronTerm(term1, alpha=a1) + 
              KronTerm(term2, alpha=a2))
        
    yerr = tt.exp(2 * logsig)
    yerr = yerr[:, None] * tt.ones(len(t))
        
    gp = xo.gp.GP(kernel, t, yerr, J=4, mean=KronMean(mean))
    gp.marginal("gp", observed = obs.T)
        
    start = model.test_point
    start["logr"] = np.log(0.01)
    start["t0"] = 0.0
    #map_soln = xo.optimize(start=start, verbose=True)
    #start = map_soln
    
    # comment this out if you don't want to run the mcmc right now.
    trace3 = pm.sample(
        tune=2000,
        draws=2000,
        start=start,
        cores=2,
        chains=2,
        step=xo.get_dense_nuts_step(target_accept=0.9)
    )

In [ ]:
import corner
samples = pm.trace_to_dataframe(trace3)
_ = corner.corner(samples)

In [ ]:
np.exp(-4.71)

In [ ]:
mean = (planet + moon)
obs = np.mean([r, g, b], axis=0) + mean

with pm.Model() as model:
    
    logS0 = pm.MvNormal("logS0", mu=mu["logS0"], 
                        chol=np.diag(sig["logS0"]), shape=2)
    logw = pm.MvNormal("logw", mu=mu["logw"], 
                       chol=np.diag(sig["logw"]), shape=2)
    logQ = pm.Normal("logQ", mu=mu["logQ"], sd=sig["logQ"])
    logsig = pm.Normal("logsig", mu=-4-np.log(3), sd=0.1)
    mean = pm.Normal("mean", mu=0.0, sd=0.1)
    u = MvUniform("u", lower=[0, 0], upper=[1, 1], testval=[0.5, 0.5])
    logr = pm.Uniform("logr", lower=-20.0, upper=0.0, testval=np.log(0.01))
    t0 = pm.Uniform("t0", lower=-1, upper=1, testval=0.0)
    
    orbit = xo.orbits.KeplerianOrbit(period=5.0)
    mean = (xo.LimbDarkLightCurve(u)
          .get_light_curve(orbit=orbit, r=tt.exp(logr), t=t-t0, texp=0.02)
          * 1e3).T
    
    term1 = xo.gp.terms.SHOTerm(
        log_S0 = logS0[0],
        log_w0 = logw[0],
        log_Q = -np.log(np.sqrt(2))
    )
    term2 = xo.gp.terms.SHOTerm(
        log_S0 = logS0[1],
        log_w0 = logw[1],
        log_Q = logQ
    )
        
    kernel = term1 + term2
        
    yerr = tt.exp(2 * logsig)
    yerr = yerr * tt.ones(len(t))
        
    gp = xo.gp.GP(kernel, t, yerr, J=4, mean=mean)
    gp.marginal("gp", observed = obs.T)
        
    start = model.test_point
    start["logr"] = np.log(0.01)
    start["t0"] = 0.0
    #map_soln = xo.optimize(start=start, verbose=True)
    #start = map_soln
    
    # comment this out if you don't want to run the mcmc right now.
    trace1 = pm.sample(
        tune=2000,
        draws=2000,
        start=start,
        cores=2,
        chains=2,
        step=xo.get_dense_nuts_step(target_accept=0.9)
    )

In [ ]:
samples = pm.trace_to_dataframe(trace1)
_ = corner.corner(samples)

In [ ]:
pl.hist(trace1.get_values('logr'), density=True, color=colors[0], 
        bins=20, histtype='step', linewidth=3, label="one band");
pl.hist(trace3.get_values('logr'), density=True, color=colors[2], 
        bins=20, histtype='step', linewidth=3, label="three bands");
pl.axvline(np.log(0.009), color='k', linewidth=2, label="true value")
pl.xlabel(r"$\mathrm{Log}(r)$")
pl.ylabel("probability")
pl.legend()

In [ ]:
pl.hist(trace1.get_values('t0'), density=True, color=colors[0], 
        bins=20, histtype='step', linewidth=3, label="one band");
pl.hist(trace3.get_values('t0'), density=True, color=colors[2], 
        bins=20, histtype='step', linewidth=3, label="three bands");
pl.axvline(0, color='k', linewidth=2, label="true value")
pl.xlabel(r"$t_0$")
pl.ylabel("probability")
pl.legend()